In [1]:
import numpy as np

from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource

Use sample data from detrend.txt

In [2]:
raw_data = []
with open("denoised_func.txt") as f:
    for line in f.readlines():
        data = line.strip("\n").split(" ")
        data = [float(i) for i in data]
        raw_data.append(data)
        
fc = np.asarray(raw_data).reshape((179,179))

Since we don't have node info just make them up for now

In [3]:
nodes = ["roi%s" % (i+1) for i in range(len(fc))]

In [29]:
xname = []
yname = []
color = []
hcolor = []
alpha = []
for i, node1 in enumerate(nodes):
    for j, node2 in enumerate(nodes):
        xname.append(node1)
        yname.append(node2)
        alpha.append(min(abs(fc[i,j]), 0.9) + 0.1)

        if fc[i,j] < -0.2:
            color.append("#eb5e60")
            hcolor.append("#e31a1c")
        elif fc[i,j] > 0.2:
            color.append("#8fbbd9")
            hcolor.append("#1f78b4")
        else:
            color.append('lightgrey')
            hcolor.append('lightgrey')

source = ColumnDataSource(data=dict(
        xname=xname,
        yname=yname,
        colors=color,
        hcolors=hcolor,
        alphas=alpha,
        count=fc.flatten(),
))

In [30]:
print ("val: %s, color: %s, hcolor: %s"%(fc[0,1],color[1],hcolor[1]))


val: -0.117696130636, color: lightgrey, hcolor: lightgrey


In [41]:
p = figure(title="Functional Connectivity Matrix",
           x_axis_location="above", tools="resize,box_zoom,hover,save,reset",
           x_range=list(reversed(nodes)), y_range=nodes)

In [42]:
p.plot_width = 800
p.plot_height = 800
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

In [43]:
rc = p.rect('xname', 'yname', 0.9, 0.9, source=source,
        color='colors', alpha='alphas', line_color=None, 
        hover_line_color="black", hover_color='hcolors', hover_alpha=1.0)

p.select_one(HoverTool).tooltips = [('names', '@yname, @xname'),
                                  ('FC strength', '@count'),]

In [44]:
output_file("fc_test.html", title="FC Matrix example")

show(p) # show the plot

Try a chord diagram

In [53]:
from util.connectome import draw_circle_points, connectomme_lines

In [48]:
lines = np.array(connectomme_lines(fc)).T

In [54]:
dot_positions = draw_circle_points(fc.shape[0])

In [76]:
pos_x=[]
pos_y=[]
for (x,y) in dot_positions:
    pos_x.append(x)
    pos_y.append(y)
    

In [139]:
beziers = ColumnDataSource({
            'x0' : lines[0][0],
            'y0' : lines[0][1],
            'x1' : lines[1][0],
            'y1' : lines[1][1],
            'cx0' : lines[0][0]/1.5,
            'cy0' : lines[0][1]/1.5,
            'cx1' : lines[1][0]/1.5,
            'cy1' : lines[1][1]/1.5
        })
 
dots = ColumnDataSource(
        data=dict(
            x=pos_x,
            y=pos_y,
            colors='#12486c',
            hover_colors='firebrick',
            desc=nodes
        ))

In [140]:
hover_dots = HoverTool(
                tooltips=[("ROI", " @desc")],
                point_policy="snap_to_data")

In [121]:
flat=fc.flatten()
width=[abs(val)*0.1 if abs(val)>0.8 else 0 for val in flat]

In [137]:
pos_y[-5:]

[-0.1746083261315511,
 -0.13994550271023357,
 -0.10511026707489396,
 -0.0701455360752043,
 -0.03509438609852564]

In [58]:
from bokeh.models import Range1d

In [141]:
p2 = figure(title="Connectomme", tools="resize,save,reset")
p2.x_range = Range1d(-1.1, 1.1)
p2.y_range = Range1d(-1.1, 1.1)
p2.bezier('x0', 'y0', 'x1', 'y1', 'cx0', 'cy0', 'cx1', 'cy1',
          source=beziers,
          line_cap='round',
          line_width=0,
          line_color="lightgrey")
p2.circle('x', 'y', size=8, line_color=None, source=dots)
p2.add_tools(hover_dots)

In [142]:
output_file("fc_conn_test.html", title="FC connectome example")

show(p2) # show the plot